In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="IGG8TZyvElUMNWRoIZXf")
project = rf.workspace("gduthwd-phkwd").project("gdut-hwd")
version = project.version(1)
dataset = version.download("coco")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 62.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to GDUT-HWD-1 in coco:: 100%|██████████| 13507/13507 [00:05<00:00, 2480.99it/s]


In [ ]:
import os

print("Dataset location:", dataset.location)
print("Folders inside:")
!ls "{dataset.location}"


Dataset location: /content/GDUT-HWD-1
Folders inside:
README.dataset.txt  README.roboflow.txt  test  train  valid


In [ ]:
import os
import json
import cv2
from tqdm import tqdm

base_dir = dataset.location        # e.g. "/content/GDUT-HWD-1"
splits = ["train", "valid"]        # we'll build train/val for classifier

out_base = "/content/helmet_cls"   # new classification dataset root

classes = ["no_helmet", "helmet"]

for split in splits:
    for c in classes:
        os.makedirs(os.path.join(out_base, split, c), exist_ok=True)

    anno_path = os.path.join(base_dir, split, "_annotations.coco.json")
    print(f"Processing {anno_path}")

    with open(anno_path, "r") as f:
        coco = json.load(f)

    cat_id_to_name = {c["id"]: c["name"] for c in coco["categories"]}
    print("Categories:", cat_id_to_name)

    # group annotations by image
    anns_by_img = {}
    for ann in coco["annotations"]:
        anns_by_img.setdefault(ann["image_id"], []).append(ann)

    img_id_to_info = {img["id"]: img for img in coco["images"]}

    for img_id, img_info in tqdm(img_id_to_info.items()):
        img_path = os.path.join(base_dir, split, img_info["file_name"])
        img = cv2.imread(img_path)
        if img is None:
            continue

        anns = anns_by_img.get(img_id, [])

        head_boxes = [a for a in anns if cat_id_to_name[a["category_id"]] == "head"]
        helmet_boxes = [a for a in anns if cat_id_to_name[a["category_id"]] == "helmet"]

        # for each head, see if any helmet overlaps => label "helmet"
        for idx, head in enumerate(head_boxes):
            x, y, w, h = head["bbox"]
            x1, y1, x2, y2 = int(x), int(y), int(x + w), int(y + h)

            label = "no_helmet"

            for helmet in helmet_boxes:
                hx, hy, hw, hh = helmet["bbox"]
                hx1, hy1, hx2, hy2 = int(hx), int(hy), int(hx + hw), int(hh + hy)

                ix1, iy1 = max(x1, hx1), max(y1, hy1)
                ix2, iy2 = min(x2, hx2), min(y2, hy2)

                if ix2 > ix1 and iy2 > iy1:
                    inter = (ix2 - ix1) * (iy2 - iy1)
                    union = (x2 - x1) * (y2 - y1) + (hx2 - hx1) * (hy2 - hy1) - inter
                    iou = inter / union if union > 0 else 0

                    if iou > 0.1:
                        label = "helmet"
                        break

            crop = img[y1:y2, x1:x2]
            if crop.size == 0:
                continue

            out_dir = os.path.join(out_base, split, label)
            out_name = f"{img_id}_{idx}.jpg"
            cv2.imwrite(os.path.join(out_dir, out_name), crop)

print("Done. Crops saved under:", out_base)
!tree -L 3 {out_base}


Processing /content/GDUT-HWD-1/train/_annotations.coco.json
Categories: {0: 'helmet', 1: 'head', 2: 'helmet', 3: 'person'}


100%|██████████| 12749/12749 [00:29<00:00, 437.24it/s]


Processing /content/GDUT-HWD-1/valid/_annotations.coco.json
Categories: {0: 'helmet', 1: 'head', 2: 'helmet', 3: 'person'}


100%|██████████| 500/500 [00:01<00:00, 478.41it/s]


Done. Crops saved under: /content/helmet_cls
/bin/bash: line 1: tree: command not found


In [ ]:
import os

base = "/content/helmet_cls"
for split in ["train", "valid"]:
    print(f"\n=== {split} ===")
    for cls in os.listdir(os.path.join(base, split)):
        cls_dir = os.path.join(base, split, cls)
        if os.path.isdir(cls_dir):
            count = len(os.listdir(cls_dir))
            print(f"{cls}: {count} images")



=== train ===
no_helmet: 13690 images
helmet: 402 images

=== valid ===
no_helmet: 642 images
helmet: 10 images


In [ ]:
import os
import random
import shutil

base = "/content/helmet_cls"

def downsample_class(split, cls_name, target_count):
    cls_dir = os.path.join(base, split, cls_name)
    all_files = [os.path.join(cls_dir, f) for f in os.listdir(cls_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

    print(f"{split}/{cls_name} has {len(all_files)} images, target = {target_count}")

    if len(all_files) <= target_count:
        print("Nothing to downsample.")
        return

    # keep this many
    keep_files = set(random.sample(all_files, target_count))

    # move the rest to a backup folder
    backup_dir = os.path.join(base, f"{split}_{cls_name}_unused")
    os.makedirs(backup_dir, exist_ok=True)

    removed = 0
    for f in all_files:
        if f not in keep_files:
            shutil.move(f, os.path.join(backup_dir, os.path.basename(f)))
            removed += 1

    print(f"Moved {removed} images from {split}/{cls_name} to {backup_dir}")

# downsample train & valid no_helmet
downsample_class("train", "no_helmet", 800)
downsample_class("valid", "no_helmet", 100)


train/no_helmet has 13690 images, target = 800
Moved 12890 images from train/no_helmet to /content/helmet_cls/train_no_helmet_unused
valid/no_helmet has 642 images, target = 100
Moved 542 images from valid/no_helmet to /content/helmet_cls/valid_no_helmet_unused


In [ ]:
import os
import random
import shutil

base = "/content/helmet_cls"

train_helmet = os.path.join(base, "train", "helmet")
valid_helmet = os.path.join(base, "valid", "helmet")

train_files = [os.path.join(train_helmet, f) for f in os.listdir(train_helmet) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

print("Train helmet before:", len(train_files))

move_count = min(60, len(train_files) - 20)  # keep at least ~20 in train
files_to_move = random.sample(train_files, move_count)

for f in files_to_move:
    shutil.move(f, os.path.join(valid_helmet, os.path.basename(f)))

print(f"Moved {move_count} helmet images to valid.")
print("Train helmet after:", len(os.listdir(train_helmet)))
print("Valid helmet now:", len(os.listdir(valid_helmet)))


Train helmet before: 402
Moved 60 helmet images to valid.
Train helmet after: 342
Valid helmet now: 70


In [ ]:
import os

base = "/content/helmet_cls"
for split in ["train", "valid"]:
    print(f"\n=== {split} ===")
    for cls in os.listdir(os.path.join(base, split)):
        cls_dir = os.path.join(base, split, cls)
        if os.path.isdir(cls_dir):
            count = len(os.listdir(cls_dir))
            print(f"{cls}: {count} images")



=== train ===
no_helmet: 800 images
helmet: 342 images

=== valid ===
no_helmet: 100 images
helmet: 70 images


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

data_dir = "/content/helmet_cls"
train_dir = f"{data_dir}/train"
val_dir   = f"{data_dir}/valid"

# 1. Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# 2. Datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset   = datasets.ImageFolder(val_dir, transform=transform)

print("Classes:", train_dataset.classes)  # should be ['helmet', 'no_helmet']

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# 3. Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 4. Model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # helmet / no_helmet

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 5. Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

    val_acc = 100.0 * correct / total if total > 0 else 0.0

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {avg_loss:.4f} - Val Acc: {val_acc:.2f}%")

# 6. Save model + class names
torch.save({
    "state_dict": model.state_dict(),
    "classes": train_dataset.classes
}, "/content/resnet18_helmet_binary.pth")

print("Saved model to /content/resnet18_helmet_binary.pth")


Classes: ['helmet', 'no_helmet']
Using device: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 182MB/s]


Epoch [1/10] - Loss: 0.4959 - Val Acc: 76.47%
Epoch [2/10] - Loss: 0.1538 - Val Acc: 80.59%
Epoch [3/10] - Loss: 0.0402 - Val Acc: 78.82%
Epoch [4/10] - Loss: 0.0163 - Val Acc: 77.65%
Epoch [5/10] - Loss: 0.0101 - Val Acc: 82.35%
Epoch [6/10] - Loss: 0.0105 - Val Acc: 77.06%
Epoch [7/10] - Loss: 0.0065 - Val Acc: 77.65%
Epoch [8/10] - Loss: 0.0054 - Val Acc: 78.24%
Epoch [9/10] - Loss: 0.0033 - Val Acc: 79.41%
Epoch [10/10] - Loss: 0.0043 - Val Acc: 80.00%
Saved model to /content/resnet18_helmet_binary.pth


In [ ]:
from google.colab import files
files.download("/content/resnet18_helmet_binary.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>